<a href="https://colab.research.google.com/github/KoganTheDev/cloud-computing-project/blob/main/input_and_output_from_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase

In [6]:
import time
!pip install firebase
from firebase import firebase
from datetime import datetime
from zoneinfo import ZoneInfo  # Python 3.9+
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import random

# Connect to Firebase
FBconn = firebase.FirebaseApplication(
    'https://cloudteamwolf-default-rtdb.europe-west1.firebasedatabase.app', None
)

local_tz = ZoneInfo("Asia/Jerusalem")

def generate_logical_data_points(n=20):
    """
    Generate n logical data points simulating slight fluctuations.
    """
    data_points = []

    base_humidity = 41.16
    base_temperature = 23.2
    base_pressure = 1
    base_distance = 50

    for i in range(n):
        # Simulate small random fluctuations
        humidity = base_humidity + random.uniform(-0.2, 0.2)
        temperature = base_temperature + random.uniform(-0.05, 0.05)
        pressure = base_pressure + random.uniform(-0.1, 0.1)
        distance = base_distance + random.uniform(-50, 50)

        point = {
            "Humidity": round(humidity, 2),
            "Temperature": round(temperature, 2),
            "Pressure": round(pressure, 2),
            "Distance": round(distance, 2)
        }
        data_points.append(point)

    return data_points

def add_data_points(data_points_raw, delay=1):
    for point in data_points_raw:
        now = datetime.now(local_tz)
        timestamp = int(time.time())

        # Separate date and time strings (no milliseconds)
        date_str = now.strftime("%Y-%m-%d")
        time_str = now.strftime("%H:%M:%S")

        # Clamp temperature between 20 and 25
        temp = max(20.0, min(25.0, point["Temperature"]))
        humidity = point["Humidity"]
        pressure = point["Pressure"]
        distance = point["Distance"]

        indoor_data = {
            "Temperature": round(temp, 2),
            "Humidity": round(humidity, 2),
            "Pressure": round(pressure, 2),
            "Distance": round(distance, 2),
            "Date": date_str,
            "Time": time_str
        }

        outdoor_temp = max(20.0, min(25.0, temp + random.uniform(0.5, 1.5)))
        outdoor_humidity = max(30.0, min(50.0, humidity - random.uniform(1.0, 3.0)))
        dlight = random.randint(600, 800)

        outdoor_data = {
            "Temperature": round(outdoor_temp, 2),
            "Humidity": round(outdoor_humidity, 2),
            "DLight": dlight,
            "Date": date_str,
            "Time": time_str
        }

        FBconn.put("Data/indoor", timestamp, indoor_data)
        FBconn.put("Data/outdoor", timestamp, outdoor_data)

        print(f"Uploaded indoor and outdoor data at timestamp: {timestamp}")

        time.sleep(delay)



def delete_data_point(key):
    """
    Delete data point with the given key from 'Data'
    """
    result = FBconn.delete('Data', key)
    print(f"Deleted data point with ID: {key}")
    return result

def get_data_points():
    """
    Retrieve all data points from 'Data'
    """
    data = FBconn.get('Datas', None)
    print("Retrieved data points:")
    print(data)
    return data

def plot_attributes_over_time(data):
    """
    data: dict from Firebase, keys are unique IDs, values are attribute dicts
    """
    if not data:
        print("No data to plot")
        return

    times = []
    humidity = []
    temperature = []
    pressure = []
    distance = []

    entries = []
    for key, val in data.items():
        # Parse timestamp with milliseconds
        time_obj = datetime.strptime(val['Time'], "%H:%M:%S.%f")
        entries.append((time_obj, val))

    entries.sort(key=lambda x: x[0])

    for time_obj, val in entries:
        times.append(time_obj)
        humidity.append(val.get('Humidity'))
        temperature.append(val.get('Temperature'))
        pressure.append(val.get('Pressure'))
        distance.append(val.get('Distance'))

    def format_axis(ax, ydata, color, ylabel):
        ax.plot(times, ydata, marker='o', color=color)
        ax.set_title(f"{ylabel} over Time")
        ax.set_xlabel("Time")
        ax.set_ylabel(ylabel)
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S.%f'))
        ax.xaxis.set_major_locator(mdates.AutoDateLocator())
        plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

        y_min = min(ydata) if ydata else 0
        y_max = max(ydata) if ydata else 1
        y_range = y_max - y_min
        if y_range == 0:
            y_range = 1
        ax.set_ylim(y_min - 0.1 * y_range, y_max + 0.1 * y_range)

    plt.figure(figsize=(12, 8))

    ax1 = plt.subplot(2, 2, 1)
    format_axis(ax1, humidity, 'b', 'Humidity')

    ax2 = plt.subplot(2, 2, 2)
    format_axis(ax2, temperature, 'r', 'Temperature')

    ax3 = plt.subplot(2, 2, 3)
    format_axis(ax3, pressure, 'g', 'Pressure')

    ax4 = plt.subplot(2, 2, 4)
    format_axis(ax4, distance, 'm', 'Distance')

    plt.tight_layout()
    plt.show()

# Generate 20 logical data points
data_points_raw = generate_logical_data_points(20)

# Add data points to Firebase
uploaded_keys = add_data_points(data_points_raw, delay=1)  # delay of 1 sec between uploads

# Retrieve all data points from Firebase
all_data = get_data_points()

# Plot the attributes over time
plot_attributes_over_time(all_data)


Uploaded indoor and outdoor data at timestamp: 1748193105
Uploaded indoor and outdoor data at timestamp: 1748193107
Uploaded indoor and outdoor data at timestamp: 1748193109
Uploaded indoor and outdoor data at timestamp: 1748193111
Uploaded indoor and outdoor data at timestamp: 1748193113
Uploaded indoor and outdoor data at timestamp: 1748193115
Uploaded indoor and outdoor data at timestamp: 1748193118
Uploaded indoor and outdoor data at timestamp: 1748193120
Uploaded indoor and outdoor data at timestamp: 1748193122
Uploaded indoor and outdoor data at timestamp: 1748193123
Uploaded indoor and outdoor data at timestamp: 1748193125
Uploaded indoor and outdoor data at timestamp: 1748193127
Uploaded indoor and outdoor data at timestamp: 1748193129
Uploaded indoor and outdoor data at timestamp: 1748193131
Uploaded indoor and outdoor data at timestamp: 1748193132
Uploaded indoor and outdoor data at timestamp: 1748193134
Uploaded indoor and outdoor data at timestamp: 1748193137
Uploaded indoo